In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib
from sklearn import cross_validation
from sklearn import grid_search

from sklearn import ensemble

In [2]:
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'rfc-tuned'

In [3]:
training_features = pd.read_csv('data/training_features.csv')
test_features = pd.read_csv('data/test_features.csv')

In [ ]:
X_full = training_features.ix[:,2:]
y_full = training_features.SeriousDlqin2yrs

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y_full, test_size=0.2, stratify=y_full, random_state=42)

In [ ]:
'''clf = joblib.load('models/rfc-2017-01-07-20-54-40.pkl')'''

clf = ensemble.RandomForestClassifier(n_jobs=-1, random_state=42)

#tuned_parameters = [{'n_estimators': [10, 25, 50, 100, 250, 500, 1000]}]
tuned_parameters = [{'n_estimators': [1000, 1500, 2000, 3000, 5000]}]

clf = grid_search.GridSearchCV(clf, tuned_parameters, cv=5, scoring='roc_auc', verbose=10)
clf.fit(X_train_full, y_train_full)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=1000 ...............................................
[CV] ...................... n_estimators=1000, score=0.858641 -  32.6s
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:   32.6s


[CV] ...................... n_estimators=1000, score=0.847258 -  33.7s
[CV] n_estimators=1000 ...............................................
[CV] ...................... n_estimators=1000, score=0.839012 -  34.1s
[CV] n_estimators=1000 ...............................................
[CV] ...................... n_estimators=1000, score=0.842395 -  34.1s
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:  2.2min


[CV] ...................... n_estimators=1000, score=0.848850 -  33.2s
[CV] n_estimators=1500 ...............................................
[CV] ...................... n_estimators=1500, score=0.858436 -  49.7s
[CV] n_estimators=1500 ...............................................
[CV] ...................... n_estimators=1500, score=0.848059 -  49.5s
[CV] n_estimators=1500 ...............................................


...

[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:  9.2min


[CV] ...................... n_estimators=2000, score=0.839590 - 1.1min
[CV] n_estimators=2000 ...............................................
[CV] ...................... n_estimators=2000, score=0.842775 - 1.1min
[CV] n_estimators=2000 ...............................................
[CV] ...................... n_estimators=2000, score=0.848597 - 1.1min
[CV] n_estimators=3000 ...............................................
[CV] ...................... n_estimators=3000, score=0.858547 - 1.7min
[CV] n_estimators=3000 ...............................................
[CV] ...................... n_estimators=3000, score=0.848290 - 1.7min
[CV] n_estimators=3000 ...............................................


[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed: 15.9min


[CV] ...................... n_estimators=3000, score=0.839641 - 1.7min
[CV] n_estimators=3000 ...............................................
[CV] ...................... n_estimators=3000, score=0.843121 - 1.7min
[CV] n_estimators=3000 ...............................................
[CV] ...................... n_estimators=3000, score=0.849178 - 1.7min
[CV] n_estimators=5000 ...............................................
[CV] ...................... n_estimators=5000, score=0.858654 - 2.8min
[CV] n_estimators=5000 ...............................................
[CV] ...................... n_estimators=5000, score=0.848248 - 2.8min
[CV] n_estimators=5000 ...............................................
[CV] ...................... n_estimators=5000, score=0.839823 - 2.8min
[CV] n_estimators=5000 ...............................................
[CV] ...................... n_estimators=5000, score=0.843528 - 2.8min
[CV] n_estimators=5000 ...............................................


[Parallel(n_jobs=1)]: Done  24 tasks       | elapsed: 32.2min


In [ ]:
clf.best_params_

In [ ]:
'''flename = 'models/%s-%s.pkl' % (model_name, timestamp)

joblib.dump(rfc, flename)

log = '%s,%s\n' % (flename, rfc)

fle = open('model_log.csv', 'a')
fle.write(log)
fle.flush()
fle.close()

print flename'''

In [ ]:
X_submission = test_features.ix[:,2:]
y_submission = clf.predict_proba(X_submission.astype(np.float32))

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test_features.ID.astype(np.int)
submission['Probability'] = y_submission[:,1]
submission.head()

In [ ]:
flename = 'submissions/submission-%s-%s.csv' % (model_name, timestamp)
submission.to_csv(flename, index=False)
print flename